# Exercise 06. End-to-end demo: Learning a Multiplayer Game using RLlib, Ray Tune, and Ray Serve

© 2019-2022, Anyscale. All Rights Reserved <br>
📖 [Back to Table of Contents](./ex_00_rllib_notebooks_table_of_contents.ipynb)<br>
⬅️ [Previous notebook](./ex_05_rllib_and_ray_serve.ipynb) <br>

This end-to-end example shows off what you will build in class.




In [1]:
from multi_agent_arena.multi_agent_arena import MultiAgentArena